In [84]:
from bayes_opt import BayesianOptimization
import pandas as pd
import lightgbm
import scipy
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from bayes_opt import UtilityFunction
from sklearn.linear_model import LinearRegression

In [53]:
SUBFOLDER = "magnets_20k_10_runs_1000_samples_v0"

In [57]:
best_features1 = pd.read_csv(f"../../data/{SUBFOLDER}/best_features_2.csv", index_col=0)
best_features1

,features
0,Feature_19633
1,Feature_4607
2,Feature_19114
3,Feature_12446
4,Feature_5098
...,...
97,Feature_686
98,Feature_18200
99,Feature_13060
100,Feature_15175


In [58]:
magnet_dataset = pd.read_csv(f"../../data/{SUBFOLDER}/magnet_dataset_x_positions_2.csv", index_col=0)
magnet_dataset

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_19991,Feature_19992,Feature_19993,Feature_19994,Feature_19995,Feature_19996,Feature_19997,Feature_19998,Feature_19999,phenotype
0,-21.156000,-30.550660,-26.379143,-27.732716,-3.190618,-5.781412,2.284280,-39.786599,4.860438,-35.871952,...,-19.288219,-4.639316,-36.857391,-16.912799,-2.433693,-36.854276,-25.548316,-1.114608,-6.523832,0.003564
1,-21.932152,-36.981035,-37.267921,-26.970943,-33.161117,-20.050130,-7.701917,3.392135,-36.305144,-36.750769,...,-16.083335,-36.894652,-18.859150,-17.636626,-17.311607,-0.440347,-11.445010,-37.540031,2.203051,0.002198
2,-5.948188,-0.534663,-25.287755,-6.440846,-16.276023,-4.680414,-17.665658,-30.758197,-10.115618,-36.944388,...,-19.730772,-20.720806,-6.646665,-15.451714,-26.867948,-11.239705,-35.675913,-36.158013,-27.383689,0.035474
3,3.486634,-29.135326,-38.196784,1.000119,-6.982979,-18.189573,-11.542605,-28.936635,-17.643718,-19.265878,...,-13.913020,-25.635236,-13.374408,-15.196573,-32.483570,-5.731413,-27.919679,-4.468089,-33.749059,0.028083
4,-30.909948,-33.157469,-23.422595,-6.712238,-31.104499,-26.556213,-9.602147,-14.361718,-11.424752,-22.259061,...,3.232664,-26.631280,-21.267242,-22.465838,-29.084002,-31.185944,-16.766164,0.314557,-31.466916,0.002497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-15.264290,-24.941704,-38.330670,-29.740570,-1.928144,-4.816506,-25.443672,-2.847306,-11.276902,-7.479122,...,0.614870,-32.856868,0.323105,-10.356023,0.731216,-29.904777,-7.335698,-27.691560,-35.385847,0.013103
996,-4.818152,-37.916910,-21.139701,-8.093301,-16.211603,-30.549824,-35.892934,-10.145146,-24.315038,-39.937300,...,3.046278,-30.673094,-1.398804,-32.723849,-21.163686,-15.466979,-32.573691,3.430893,2.489943,0.002549
997,-4.113591,3.380258,-34.139308,-0.317290,4.287990,-22.571291,-29.152136,3.934148,-26.954644,-3.553665,...,-28.751247,-5.754346,-39.911010,0.997833,-11.791650,-28.875762,-3.563498,-13.496933,-7.019844,0.000000
998,-39.667487,-29.704773,-38.136840,-14.469638,-23.964862,-38.019510,-16.643283,-38.601006,0.286321,-34.139550,...,-15.976286,-30.986734,-7.778806,-3.947261,-14.439739,-8.232817,-6.138125,-37.250640,-29.753095,0.003545


In [59]:
best_features = pd.read_csv(f"../../data/{SUBFOLDER}/best_features_2.csv", index_col=0)
list_best_features = list(best_features["features"].values)
list_best_features

['Feature_19633',
 'Feature_4607',
 'Feature_19114',
 'Feature_12446',
 'Feature_5098',
 'Feature_13476',
 'Feature_10410',
 'Feature_17804',
 'Feature_6180',
 'Feature_11100',
 'Feature_189',
 'Feature_6387',
 'Feature_8026',
 'Feature_5427',
 'Feature_1394',
 'Feature_12563',
 'Feature_17820',
 'Feature_13213',
 'Feature_5936',
 'Feature_9972',
 'Feature_9539',
 'Feature_12681',
 'Feature_3174',
 'Feature_5789',
 'Feature_13820',
 'Feature_1266',
 'Feature_16658',
 'Feature_19501',
 'Feature_1918',
 'Feature_10738',
 'Feature_2974',
 'Feature_3299',
 'Feature_10614',
 'Feature_10375',
 'Feature_16380',
 'Feature_11114',
 'Feature_334',
 'Feature_15082',
 'Feature_3348',
 'Feature_1285',
 'Feature_18615',
 'Feature_13491',
 'Feature_11917',
 'Feature_17792',
 'Feature_18272',
 'Feature_11251',
 'Feature_13269',
 'Feature_19153',
 'Feature_5717',
 'Feature_15451',
 'Feature_13861',
 'Feature_4220',
 'Feature_8281',
 'Feature_15523',
 'Feature_9763',
 'Feature_5796',
 'Feature_19162',
 

In [60]:
random.sample(list_best_features, 10)

['Feature_12563',
 'Feature_19501',
 'Feature_12119',
 'Feature_5717',
 'Feature_12012',
 'Feature_18200',
 'Feature_919',
 'Feature_7961',
 'Feature_18843',
 'Feature_2879']

In [61]:
# One iter
# 50 times
    # Choose one of the datasets
    # Choose 10 features
    # Train test split
    # Predict label

In [89]:
lgb_param_space = {
    "learning_rate": (0.001, 0.5),  
    "n_estimators": (5, 1000),
    "max_depth": (3, 16),
    "min_child_samples": (5, 300),
    "subsample": (0.5, 1),  
    "reg_alpha": (0.01, 7),
    "reg_lambda": (0, 10)
}

In [93]:
def black_box_function(learning_rate, n_estimators, max_depth, min_child_samples, subsample, reg_alpha, reg_lambda):
    """Function with unknown internals we wish to maximize.

    One iteration
    20 times
        Choose one of the datasets
        Choose 10 features
        Train test split
        Predict label
    """
    scores = []
    for i in range(20):
        selected_dataset_id = 2#np.random.randint(1,11)
        best_features = pd.read_csv(f"../../data/{SUBFOLDER}/best_features_{selected_dataset_id}.csv", index_col=0)
        list_best_features = list(best_features["features"].values)
        chosen_features = random.sample(list_best_features, 10)
        magnet_dataset = pd.read_csv(f"../../data/{SUBFOLDER}/magnet_dataset_x_positions_{selected_dataset_id}.csv", index_col=0)
        y = magnet_dataset["phenotype"]
        X = magnet_dataset.drop("phenotype", axis=1)
        X = X[chosen_features]
        #print(X.shape)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        model = lightgbm.LGBMRegressor(learning_rate=learning_rate, 
                                       n_estimators=int(n_estimators),
                                       max_depth=int(max_depth),
                                       min_child_samples=int(min_child_samples),
                                       subsample=subsample,
                                       reg_alpha=reg_alpha, 
                                       reg_lambda=reg_lambda,
                                       verbose=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        scores.append(mae)
    return -np.mean(scores)


In [94]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=lgb_param_space,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [95]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | -0.008115 | 0.2091    | 12.36     | 5.034     | 305.8     | 1.036     | 0.9234    | 0.5931    |
| 2         | -0.007914 | 0.1734    | 8.158     | 164.0     | 422.1     | 4.8       | 2.045     | 0.9391    |
| 3         | -0.008016 | 0.03859   | 4.596     | 163.0     | 422.0     | 1.563     | 4.084     | 0.9567    |
| 4         | -0.008016 | 0.3828    | 4.424     | 159.0     | 769.4     | 4.813     | 1.33      | 0.601     |
| 5         | -0.007983 | 0.4825    | 3.862     | 20.41     | 613.5     | 4.058     | 8.608     | 0.7013    |


In [81]:
print(optimizer.max)

{'target': -0.007916603280439707, 'params': {'learning_rate': 0.18928429603604183, 'max_depth': 9.718725903016507, 'min_child_samples': 6.398596693425054, 'num_leaves': 79.27365182110255, 'reg_alpha': 3.4851377451572576, 'reg_lambda': 2.233256325939111, 'subsample': 0.8580475371756726}}


In [82]:
best_model = lightgbm.LGBMRegressor(**optimizer.max["params"])

In [83]:
best_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.18928429603604183,
 'max_depth': 9.718725903016507,
 'min_child_samples': 6.398596693425054,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 79.27365182110255,
 'objective': None,
 'random_state': None,
 'reg_alpha': 3.4851377451572576,
 'reg_lambda': 2.233256325939111,
 'subsample': 0.8580475371756726,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [86]:
scores = []
random_scores = []
lr_scores = []
for i in range(20):
    selected_dataset_id = 2#np.random.randint(1,11)
    best_features = pd.read_csv(f"../../data/{SUBFOLDER}/best_features_{selected_dataset_id}.csv", index_col=0)
    list_best_features = list(best_features["features"].values)
    chosen_features = random.sample(list_best_features, 10)
    magnet_dataset = pd.read_csv(f"../../data/{SUBFOLDER}/magnet_dataset_x_positions_{selected_dataset_id}.csv", index_col=0)
    y = magnet_dataset["phenotype"]
    X = magnet_dataset.drop("phenotype", axis=1)
    X = X[chosen_features]
    #print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    best_params = optimizer.max["params"]
    model = lightgbm.LGBMRegressor(learning_rate=best_params["learning_rate"], 
                                       num_leaves=int(best_params["num_leaves"]),
                                       max_depth=int(best_params["max_depth"]),
                                       min_child_samples=int(best_params["min_child_samples"]),
                                       subsample=best_params["subsample"],
                                       reg_alpha=best_params["reg_alpha"], 
                                       reg_lambda=best_params["reg_lambda"],
                                       verbose=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    scores.append(mae)
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    lr_scores.append(mae)
    y_shuffled = y.sample(frac=1)
    mae = mean_absolute_error(y_test, y_pred)
    random_scores.append(mae)

In [88]:
print(np.mean(scores), np.mean(lr_scores), np.mean(random_scores))

0.008027782318652732 0.00807777753641265 0.00807777753641265


In [9]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -7.135455292718879, 'params': {'x': 2.8340440094051482, 'y': 1.3219469606529488}}
Iteration 1: 
	{'target': -7.779531005607566, 'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}}
Iteration 2: 
	{'target': -7.109925819441113, 'params': {'x': 2.2175526295255183, 'y': -0.7867249801593896}}
Iteration 3: 
	{'target': -12.397162416009818, 'params': {'x': 3.660003815774634, 'y': 0.9608275029525108}}
Iteration 4: 
	{'target': -6.999472814518675, 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}


In [10]:
utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)
next_point_to_probe = optimizer.suggest(utility)
target = black_box_function(**next_point_to_probe)

In [ ]:
optimizer.register(
    params=next_point_to_probe,
    target=target,
)

In [ ]:
for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)

    print(target, next_point)
print(optimizer.max)